<a href="https://colab.research.google.com/github/ilgarbuzov/interview_tests/blob/master/analytics_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment description

**Prerequisites**:

1. Authorize with your Google account
2. Go to Google Cloud Console
3. Select existing Google Cloud project or create a new one
4. Enable Google BigQuery API for the selected project
5. Copy this document to your google drive

**Implementation**:
1. Complete all assignments in this document
2. Share your variant of the document with us

In [ ]:
#run this cell first
import numpy as np
import pandas as pd
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


#Assignment #1
Prepare SQL which converts raw hits (user events) into sessions.

Count the number of sessions in the result table.

**In**:
 - [Session definition](https://support.google.com/analytics/answer/2731565?hl=en)
 - Public table with samle hits data: `analytics-230012.Assignments.hits_for_sessions`
 - Use your Google Cloud project ID instead of  `project_id`

In [ ]:
query = '''
SELECT *  FROM `analytics-230012.Assignments.hits_for_sessions` #example
'''

project_id = 'animated-graph-280813' #change

df = pd.io.gbq.read_gbq(query, project_id=project_id, verbose=False, dialect='standard')
df.head(3)

,hitId,anonymousId,hitTime,utm_campaign,utm_source,utm_medium
0,0a0a29e0-0c2d-11e8-b6de-ffc22b5b4218,0129fa30-0c2d-11e8-8bc5-d9061171bfdd,2018-02-07 17:33:42.265000+00:00,None,None,None
1,0a09b4b0-0c2d-11e8-b6de-ffc22b5b4218,0129fa30-0c2d-11e8-8bc5-d9061171bfdd,2018-02-07 17:33:42.267999+00:00,None,None,None
2,12320610-0c2d-11e8-a454-493f16c454c9,0129fa30-0c2d-11e8-8bc5-d9061171bfdd,2018-02-07 17:33:55.938999+00:00,None,None,None


In [ ]:
query = '''
#Start your query here
#SELECT *  FROM `analytics-230012.Assignments.hits_for_sessions` #example

    SELECT count(*)
     FROM (
        SELECT *,
                  LAG(hitTime, 1) OVER
                      (PARTITION BY anonymousId ORDER BY hitTime) AS last_event
              FROM `analytics-230012.Assignments.hits_for_sessions`
        ) last
          WHERE TIMESTAMP_DIFF(hitTime, last_event, second) >= (60 * 30) AND TIMESTAMP_DIFF(hitTime, last_event, day) = 0
              OR last_event IS NULL

#Finish your query here
'''

project_id = 'animated-graph-280813' #change

df = pd.io.gbq.read_gbq(query, project_id=project_id, verbose=False, dialect='standard')
df.head()

,f0_
0,605


#Assignment #2
Count all the users matching the following criteria:
- Have multiple transactions
- Have at least one transaction that was made within 7 days from the previous transaction

**In**:
 - Public table with samle transactions data: `analytics-230012.Assignments.orders`
 - Use your Google Cloud project ID instead of  `project_id`

In [ ]:
query = '''
SELECT * FROM `analytics-230012.Assignments.orders` #example
'''

project_id = 'animated-graph-280813' #change

df = pd.io.gbq.read_gbq(query, project_id=project_id, verbose=False, dialect='standard')
df.head()

,userId,orderId,date
0,8,275256,2016-09-16 11:32:06
1,30,278443,2016-10-12 11:35:47
2,30,280430,2016-10-27 14:22:39
3,30,BK86661,2017-06-15 08:36:14
4,30,EDB87580-81C6-43EC-B7C0-19F1C7DB1DD4,2017-01-04 14:50:36


In [ ]:
query = '''
#Start your query here

  #SELECT * FROM `analytics-230012.Assignments.orders` #example
 
SELECT count(*) from(
    SELECT userId
          FROM (
                SELECT *, LAG(date, 1) OVER
                          (PARTITION BY userId ORDER BY date) AS lastDate
                FROM `analytics-230012.Assignments.orders` 
                )  last
          WHERE DATETIME_DIFF(date, lastDate, day) <=7
                  group by userId
)             
        

#Finish your query here
'''

project_id = 'animated-graph-280813' #change

df = pd.io.gbq.read_gbq(query, project_id=project_id, verbose=False, dialect='standard')
df.head()

,f0_
0,1624


#Assignment #3
Below you can see a matrix. We highlighted few sequences of 5 neighbour numbers in different directions. There are 4 directions: vertical, horizontal, diagonal1(left,down -> right,up), diagonal2(left,up -> right,down).
 - Product of 5 horizontal numbers example: 38 x 88 x 57 x 19 x 22 = 79674144
 - Product of 5 diagonal1 numbers example: 84 x 58 x 83 x 10 x 90 = 363938400

**What is the maximum product for any sequence?**

![alt text](https://drive.google.com/uc?export=view&id=1TGORTF_ffAeu5ujS5s461U2kkzUErN3w)

In [ ]:
def largestProduct(arr, n): 

	maxProduct = 0

	# iterate the rows. 
	for i in range(n): 

		# iterate the columns. 
		for j in range( n): 

			# check the maximum product 
			# in horizontal row. 
			if ((j - 4) >= 0): 
				result = (arr[i][j] * arr[i][j - 1] *
						arr[i][j - 2] * arr[i][j - 3] * arr[i][j - 4]) 
				
				if (maxProduct < result): 
					maxProduct = result
                    
			# check the maximum product 
			# in vertical row. 
			if ((i - 4) >= 0) : 
				result = (arr[i][j] * arr[i - 1][j] *
						arr[i - 2][j] * arr[i - 3][j] * arr[i - 4][j]) 
				
				if (maxProduct < result): 
					maxProduct = result
                    
            # check the maximum product in 
			# diagonal going through down - right 
			if ((i - 4) >= 0 and (j - 4) >= 0): 
				result = (arr[i][j] * arr[i - 1][j - 1] *
						arr[i - 2][j - 2] * arr[i - 3][j - 3] * arr[i - 4][j - 4]) 
				
				if (maxProduct < result): 
					maxProduct = result

            # check the maximum product in 
			# diagonal going through up - right 
			if ((i - 4) >= 0 and (j - 1) <= 0): 
				result = (arr[i][j] * arr[i - 1][j + 1] *
						arr[i - 2][j + 2] * arr[i - 3][j + 3]  * arr[i - 4][j + 4]) 

				if (maxProduct < result): 
					maxProduct = result
                    

	return maxProduct


matrix = [[30, 90, 80, 36, 44, 66, 42, 34, 65, 52, 88, 26, 79, 81, 55, 92, 37, 37, 4, 84],
        [93, 4, 92, 3, 37, 87, 93, 84, 94, 62, 55, 7, 88, 86, 9, 33, 70, 78, 9, 61],
        [72, 2, 2, 37, 38, 88, 57, 19, 22, 82, 71, 53, 14, 60, 73, 5, 45, 77, 76, 57],
        [32, 34, 77, 55, 39, 93, 36, 95, 65, 32, 95, 90, 53, 71, 97, 70, 85, 38, 72, 37],
        [37, 53, 33, 97, 41, 1, 77, 7, 68, 10, 1, 45, 21, 94, 9, 80, 98, 45, 89, 87],
        [19, 12, 58, 2, 27, 83, 70, 12, 75, 12, 36, 43, 15, 79, 40, 90, 91, 74, 58, 84],
        [45, 34, 34, 29, 14, 89, 51, 65, 87, 24, 78, 26, 94, 82, 64, 75, 14, 9, 76, 26],
        [86, 30, 86, 21, 30, 0, 57, 90, 11, 0, 41, 48, 8, 38, 37, 41, 85, 84, 46, 74],
        [61, 83, 37, 84, 98, 44, 10, 32, 8, 18, 68, 36, 43, 49, 26, 28, 47, 37, 81, 23],
        [39, 74, 1, 51, 73, 83, 85, 41, 82, 25, 3, 32, 16, 15, 66, 51, 37, 68, 47, 28],
        [78, 68, 27, 37, 58, 88, 63, 54, 89, 11, 56, 47, 20, 57, 58, 7, 7, 36, 26, 20],
        [11, 36, 87, 84, 82, 68, 12, 16, 37, 16, 0, 50, 96, 25, 53, 0, 27, 78, 0, 57],
        [4, 70, 6, 96, 92, 0, 96, 15, 24, 32, 13, 49, 61, 52, 30, 38, 3, 77, 43, 17],
        [45, 78, 70, 45, 10, 61, 60, 71, 0, 18, 91, 27, 0, 4, 66, 72, 7, 74, 89, 73],
        [94, 97, 29, 25, 89, 67, 91, 65, 17, 27, 51, 61, 86, 21, 95, 79, 1, 54, 89, 80],
        [59, 90, 72, 94, 60, 3, 39, 61, 69, 49, 52, 89, 50, 74, 84, 56, 50, 86, 68, 26],
        [66, 47, 67, 78, 68, 65, 20, 83, 78, 37, 19, 52, 16, 76, 12, 25, 35, 81, 26, 81],
        [67, 3, 31, 26, 27, 61, 36, 84, 19, 53, 77, 69, 40, 13, 91, 78, 0, 54, 94, 2],
        [20, 59, 70, 91, 74, 39, 90, 55, 53, 74, 12, 79, 48, 40, 1, 73, 6, 68, 56, 40],
        [7, 33, 59, 64, 84, 25, 89, 38, 19, 29, 93, 77, 46, 86, 45, 93, 63, 96, 53, 29]]
				


'''30 90 80 36 44 66 42 34 65 52 88 26 79 81 55 92 37 37 04 84
93 04 92 03 37 87 93 84 94 62 55 07 88 86 09 33 70 78 09 61
72 02 02 37 38 88 57 19 22 82 71 53 14 60 73 05 45 77 76 57
32 34 77 55 39 93 36 95 65 32 95 90 53 71 97 70 85 38 72 37
37 53 33 97 41 01 77 07 68 10 01 45 21 94 09 80 98 45 89 87
19 12 58 02 27 83 70 12 75 12 36 43 15 79 40 90 91 74 58 84
45 34 34 29 14 89 51 65 87 24 78 26 94 82 64 75 14 09 76 26
86 30 86 21 30 00 57 90 11 00 41 48 08 38 37 41 85 84 46 74
61 83 37 84 98 44 10 32 08 18 68 36 43 49 26 28 47 37 81 23
39 74 01 51 73 83 85 41 82 25 03 32 16 15 66 51 37 68 47 28
78 68 27 37 58 88 63 54 89 11 56 47 20 57 58 07 07 36 26 20
11 36 87 84 82 68 12 16 37 16 00 50 96 25 53 00 27 78 00 57
04 70 06 96 92 00 96 15 24 32 13 49 61 52 30 38 03 77 43 17
45 78 70 45 10 61 60 71 00 18 91 27 00 04 66 72 07 74 89 73
94 97 29 25 89 67 91 65 17 27 51 61 86 21 95 79 01 54 89 80
59 90 72 94 60 03 39 61 69 49 52 89 50 74 84 56 50 86 68 26
66 47 67 78 68 65 20 83 78 37 19 52 16 76 12 25 35 81 26 81
67 03 31 26 27 61 36 84 19 53 77 69 40 13 91 78 00 54 94 02
20 59 70 91 74 39 90 55 53 74 12 79 48 40 01 73 06 68 56 40
07 33 59 64 84 25 89 38 19 29 93 77 46 86 45 93 63 96 53 29'''

print(largestProduct(matrix, 20))
#type(matrix)

4703677440
